# 주택 가격 예측

- 캘리포니아 인구조사 데이터를 사용해 캘리포니아 주택 가격 모델을 만드는 것이 목표
- 이 데이터는 캘리포니아의 블록 그룹 마다 인구, 중간 소득, 중간 주택 가격 등을 담고 있다.
- 블록 그룹은 미국 인구조사국에서 샘플 데이터를 발표하는 데 사용하는 최소한의 지리적 단위
- 하나의 블록 그룹은 보통 600 ~ 3000명의 인구를 나타낸다.
- 이 데이터로 모델을 학습시켜서 다른 측정 데이터가 주어졌을 때 구역의 중간 주택 가격을 예측해보자.